## ## XGBoost Dask scaling

Distributed training using Dask and XGBoost
[Tutorial link](http://matthewrocklin.com/blog/work/2017/03/28/dask-xgboost)

In [7]:
!pip install dask_ml dask_xgboost s3fs

     |████████████████████████████████| 6.3 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 5.7 MB/s eta 0:00:01


In [8]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=1)
client

/opt/conda/envs/rapids/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37013 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:38513 Dashboard: http://127.0.0.1:37013/status,Cluster Workers: 4 Cores: 4 Memory: 540.94 GB


In [9]:
import dask.dataframe as dd
import dask
# Subset of the columns to use
cols = ['Year', 'Month', 'DayOfWeek', 'Distance',
        'DepDelay', 'CRSDepTime', 'UniqueCarrier', 'Origin', 'Dest']

# Create the dataframe
df = dd.read_csv('s3://dask-data/airline-data/20*.csv', usecols=cols, storage_options={'anon': True})

In [10]:
df = df.sample(frac=0.2) # XGBoost requires a bit of RAM, we need a larger cluster

In [12]:
is_delayed = (df.DepDelay.fillna(16) > 15)  # column of labels
del df['DepDelay']  # Remove delay information from training dataframe

df['CRSDepTime'] = df['CRSDepTime'].clip(upper=2399)

df, is_delayed = dask.persist(df, is_delayed)  # start work in the background

NameError: name 'dask' is not defined